In [ ]:
import ROOT, os

In [44]:
CR_cut = f"""(
    (((jet_mass>50) & (jet_mass<80)) | (jet_mass>150)) & 
    (tagger>0.98) & (tagger<2)
)"""

# Read file

In [ ]:
f = ROOT.TFile(f"input/Run2/data.root", "r")
tree = f.Get("Events")
fit_mass = ROOT.RooRealVar("fit_mass", "fit_mass", 1500, 650, 4000)
weight = ROOT.RooRealVar("weight", "weight", 1, -10, 10)
jet_mass = ROOT.RooRealVar("jet_mass", "jet_mass", 125, 0, 999)
tagger = ROOT.RooRealVar("tagger", "tagger", 0, 0, 2)
data_region = ROOT.RooDataSet(f"data_CR", f"data_CR", tree, ROOT.RooArgSet(fit_mass, weight, jet_mass, tagger), CR_cut, "weight")


p1 = ROOT.RooRealVar("p1", "p1", -0.1, -10, 0)
model = ROOT.RooGenericPdf("model_background_expow1", "model_background_expow1", "TMath::Power(@0, @1)", ROOT.RooArgList(fit_mass, p1))
result = model.fitTo(data_region, ROOT.RooFit.SumW2Error(True), Save=True)
p1.setConstant(True)


# Fit

In [ ]:
model, p1, p2, p3, result = {}, {}, {}, {}, {}

# dijet2 model
p1['dijet2'] = ROOT.RooRealVar("p1", "p1", 1, -10, 100)
p2['dijet2'] = ROOT.RooRealVar("p2", "p2", -1, -10, 10)
model['dijet2'] = ROOT.RooGenericPdf("model_background_dijet2", "model_background_dijet2", "TMath::Power(@0, @1 + @2 * TMath::Log(@0))", ROOT.RooArgList(fit_mass, p1['dijet2'], p2['dijet2']))

# dijet3 model
p1['dijet3'] = ROOT.RooRealVar("p1", "p1", 1, -10, 10)
p2['dijet3'] = ROOT.RooRealVar("p2", "p2", -1, -10, 10)
p3['dijet3'] = ROOT.RooRealVar("p3", "p3", -0.1, -10, 10)
model['dijet3'] = ROOT.RooGenericPdf("model_background_dijet3", "model_background_dijet3", "TMath::Power(@0, @1 + @2 * TMath::Log(@0) + @3 * TMath::Power(TMath::Log(@0), 2))", ROOT.RooArgList(fit_mass, p1['dijet3'], p2['dijet3'], p3['dijet3']))

# expow1 model
p1['expow1'] = ROOT.RooRealVar("p1", "p1", -0.1, -10, 0)
model['expow1'] = ROOT.RooGenericPdf("model_background_expow1", "model_background_expow1", "TMath::Power(@0, @1)", ROOT.RooArgList(fit_mass, p1['expow1']))
_model = ROOT.RooGenericPdf("model_background_expow1", "model_background_expow1", "TMath::Power(@0, @1)", ROOT.RooArgList(fit_mass, p1['expow1']))


# expow2 model
p1['expow2'] = ROOT.RooRealVar("p1", "p1", -0.01, -5, 0)
p2['expow2'] = ROOT.RooRealVar("p2", "p2", -0.001, -1, 0)
model['expow2'] = ROOT.RooGenericPdf("model_background_expow2", "model_background_expow2", "TMath::Power(@0, @1) * TMath::Exp(@2 * @0)", ROOT.RooArgList(fit_mass, p1['expow2'], p2['expow2']))

# invpow2 model
p1['invpow2'] = ROOT.RooRealVar("p1", "p1", 1, -1, 100)
p2['invpow2'] = ROOT.RooRealVar("p2", "p2", -1, -100, 1)
model['invpow2'] = ROOT.RooGenericPdf("model_background_invpow2", "model_background_invpow2", "TMath::Power(1 + @1*@0, @2)", ROOT.RooArgList(fit_mass, p1['invpow2'], p2['invpow2']))

# invpow3 model
p1['invpow3'] = ROOT.RooRealVar("p1", "p1", 1, -1, 100)
p2['invpow3'] = ROOT.RooRealVar("p2", "p2", -1, -100, 1)
p3['invpow3'] = ROOT.RooRealVar("p3", "p3", 1e-6, -1, 1)
model['invpow3'] = ROOT.RooGenericPdf("model_background_invpow3", "model_background_invpow3", "TMath::Power(1 + @1*@0, @2 + @3*@0)", ROOT.RooArgList(fit_mass, p1['invpow3'], p2['invpow3'], p3['invpow3']))

for k in model:
    result[k] = model[k].fitTo(data_region, ROOT.RooFit.SumW2Error(True), Save=True)
    p1[k].setConstant(True)
    if k in p2:
        p2[k].setConstant(True)
    if k in p3:
        p3[k].setConstant(True)

In [ ]:
for k in p1:
    print('p1', k, p1[k].getVal())
print()
for k in p2:
    print('p2', k, p2[k].getVal())

In [ ]:
#['expow1', 'expow2', 'dijet2', 'dijet3', 'invpow2', 'invpow3']
c = ROOT.TCanvas()
ROOT.gPad.SetLogy()

line_color = {'expow1': ROOT.kRed, 'expow2': ROOT.kYellow+1, 'dijet2': ROOT.kGreen, 'dijet3': ROOT.kCyan, 'invpow2':ROOT.kBlue, 'invpow3': ROOT.kMagenta}
band_color = {'expow1': ROOT.kPink, 'expow2': ROOT.kYellow, 'dijet2': ROOT.kGreen, 'dijet3': ROOT.kCyan, 'invpow2':ROOT.kAzure, 'invpow3': ROOT.kMagenta}

frame = fit_mass.frame(650, 4000, 67)
fit_mass.setBins(67)
data_binned = model['dijet2'].generateBinned({fit_mass}, 1000)
data_binned.plotOn(frame)

line = {}
candidates = ['expow1', 'expow2', 'dijet2', 'dijet3', 'invpow2', 'invpow3']
#for k in candidates:
#    model[k].plotOn(frame, VisualizeError=(result[k], 1), FillColor='kGray')
for k in candidates:
    model[k].plotOn(frame, LineColor=line_color[k])
data_binned.plotOn(frame)
frame.SetMinimum(1e-1)

frame.Draw()
c.SaveAs("/afs/cern.ch/user/d/dfu/NanoAnalysis/fit/test.pdf")